In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
from scipy.stats import sem
from scipy.stats import pearsonr
from scipy.optimize import curve_fit, minimize, fmin
from sklearn.metrics import r2_score
from scipy.integrate import quad

In [ ]:
def gaussian(x, μ, σ):
    return np.exp(-((x - μ) / σ)**2 / 2) / np.sqrt(2 * np.pi * σ**2)


def gaussian_pr(x, μ, σ):
    return x * gaussian(x, μ, σ)


def my_trapz(func, a, b, p0, p1, n_steps):
    X = np.linspace(a,b,n_steps)
    integral = (func(a, p0, p1)+func(b, p0, p1))/2.0 + sum([func(x, p0, p1) for x in X])
    return integral * (b-a)/n_steps
def my_trapz3(func, a, b, p0, p1,p2, n_steps):
    X = np.linspace(a,b,n_steps)
    integral = (func(a, p0, p1, p2)+func(b, p0, p1, p2))/2.0 + sum([func(x, p0, p1, p2) for x in X])
    return integral * (b-a)/n_steps


def my_trapz2(func, a, b, n_steps):
    X = np.linspace(a,b,n_steps)
    integral = (func(a)+func(b))/2.0 + sum([func(x) for x in X])
    return integral * (b-a)/n_steps




def gaussian_mrg(tm, ts, wm, tp, te, wp):
    return gaussian(tm, ts, wm*ts) * gaussian(tp, te, wp*te)


#prior is 1/tsmax-tsmin, now we are going to implement a modified prior
def sum_function(ts,td,wp,r):
    #r is learning rate
    t_min=0.4
    t_max=1
    return (1/(t_max-t_min))+r*gaussian(ts,td,wp*td)
def modified_prior(ts,td,wp,r):
    #r is learning rate
    t_min=0.4
    t_max=1
    num=(1/(t_max-t_min))+r*gaussian(ts,td,wp*td)
    f = lambda ts: sum_function(ts, td, wp, r)
    denom=my_trapz2(f,t_min,t_max,100)+(10 ** -50)
    #denom=my_trapz3(sum_function, t_min, t_max, td, wp, r, 1000) + (10 ** -50)
    return num/denom
def modified_posterior(ts,tm,td,wp,wm,r):
    nom=modified_prior(ts,td,wp,r)*gaussian(tm,ts,ts*wm)
    f = lambda ts: modified_prior(ts,td,wp,r)*gaussian(tm,ts,wm*ts)
    t_min=0.4
    t_max=1
    denom=my_trapz2(f,t_min,t_max,100)+(10 ** -50)
    return nom/denom
################################################
def BLS_modified(ts,td,wm,wp,r):
    t_min = 0.4
    t_max = 1
    #ts = np.array([0.4, 0.6, 1])
    f_num= lambda tm : tm * modified_posterior(ts,tm,td,wp,wm,r)
    BLS_num = my_trapz2(f_num, t_min, t_max, 500)
    #print(BLS_num)
    f_denom= lambda tm : modified_posterior(ts,tm,td,wp,wm,r)
    BLS_denom = my_trapz2(f_denom, t_min, t_max, 100) + (10 ** -50)
    #print(BLS_denom)
    return BLS_num / BLS_denom


def Bayes_model(p, tp_dist):
    # tp dist has 9 conds (ts,td)  (0.4,0.5)     (0.6,0.5)     (1,0.5)
    #                              (0.4,0.9)     (0.6,0.9)     (1,0.9)
    #                              (0.4,1.7)     (0.6,1.7)     (1,1.7)
    t_min = 0.4
    t_max = 1
    ts = np.array([0.4, 0.6, 1])
    td=np.array([0.5,0.9,1.7])
    new_te=np.empty((len(ts),len(td))) * np.nan
    for i in range(len(ts)):
        for j in range(len(td)):
            new_te[i,j]=BLS_modified(ts[i],td[j],p[0],p[1],p[2])

    logLike = np.empty((len(ts),len(td))) * np.nan
    for i in range(len(ts)):
        for j in range(len(td)):

            f = lambda tm: gaussian_mrg(tm, ts[i], p[0], tp_dist[i][j], new_te[i,j], p[1])
            logLike[i,j] = np.sum(np.log(my_trapz2(f, -10, 10, 100)))
    return -np.sum(logLike.flatten())


def Bayes_data(p):
    t_min = 0.4
    t_max = 1
    
    td=np.array([0.5,0.9,1.7])
    ts = np.array([0.4, 0.6,1])
    new_te=np.empty((len(ts),len(td))) * np.nan
    for i in range(len(ts)):
        for j in range(len(td)):
            new_te[i,j]=BLS_modified(ts[i],td[j],p[0],p[1],p[2])


    tp_dist = np.linspace(0, 3, 1000)
    μ = np.empty((len(ts),len(td))) * np.nan 
    σ = np.empty((len(ts),len(td))) * np.nan
    lhood= np.empty((len(ts),len(td))) * np.nan
    for i in range(len(ts)):
        for j in range(len(td)):
            f = lambda tm: gaussian_mrg(tm, ts[i], p[0], tp_dist, new_te[i,j], p[1])
            likelihood = my_trapz2(f, -10, 10, 1000)
            likelihood /= likelihood.sum()
            dist = np.random.choice(tp_dist, p=likelihood, size=10000)
            μ[i,j] = np.mean(dist)
            σ[i,j] = np.std(dist)
            #lhood[i,j]=likelihood
    return μ, σ

In [ ]:
def fitter_double_prior(dir):
    df=pd.read_csv(dir)
    df=df.dropna()
    t_p400_500=np.array(df[df['random_gaps']==0.5].query('t_sample==0.4')['t_prod'])
    t_p600_500=np.array(df[df['random_gaps']==0.5].query('t_sample==0.6')['t_prod'])
    t_p1000_500=np.array(df[df['random_gaps']==0.5].query('t_sample==1')['t_prod'])


    t_p400_900=np.array(df[df['random_gaps']==0.9].query('t_sample==0.4')['t_prod'])
    t_p600_900=np.array(df[df['random_gaps']==0.9].query('t_sample==0.6')['t_prod'])
    t_p1000_900=np.array(df[df['random_gaps']==0.9].query('t_sample==1')['t_prod'])


    t_p400_1700=np.array(df[df['random_gaps']==1.7].query('t_sample==0.4')['t_prod'])
    t_p600_1700=np.array(df[df['random_gaps']==1.7].query('t_sample==0.6')['t_prod'])
    t_p1000_1700=np.array(df[df['random_gaps']==1.7].query('t_sample==1')['t_prod'])



    t_ps=[[t_p400_500,t_p400_900,t_p400_1700],
      [t_p600_500,t_p600_900,t_p600_1700],
      [t_p1000_500,t_p1000_900,t_p1000_1700]]

    g = np.array([0.3,0.3,0.3])
    bnds = ((0.001, 1), (0.001, 1,(0.001, 1))
    res=minimize(Bayes_model, g, args=(t_ps), bounds=bnds,
                method='Nelder-Mead', options={'gtol': 1e-6, 'disp': True})
    print(res)
    print('######################################################################')
